# Import everything

In [1]:
import glob
import pandas as pd
import mne
import matplotlib.pyplot as plt
from io import StringIO
import mne
from mne.io import read_raw_eeglab, read_epochs_eeglab
import numpy as np
from scipy import signal
from scipy import fftpack
import seaborn as sns
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm
import math
from sklearn import preprocessing
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from keras_tqdm import TQDMNotebookCallback
from tensorboard.plugins.hparams import api as hp
from livelossplot.tf_keras import PlotLossesCallback

import sklearn.model_selection
import sklearn.datasets
import sklearn.metrics
import multiprocessing
from tpot import TPOTRegressor
from oct2py import octave
from joblib import Parallel, delayed
import multiprocessing
from joblib import wrap_non_picklable_objects
import json
import pickle
import os.path
from mpl_toolkits.mplot3d import axes3d
import timeit
from skimage.transform import resize
from timeit import default_timer as timer
from datetime import timedelta
import json
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, callbacks
from tensorflow.keras.models import Model, load_model
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras.utils import plot_model
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.genmod import bayes_mixed_glm as glm
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import RFECV
from spectrum import arburg, arma2psd, pburg
import pylab
from scipy.signal import find_peaks, butter
from scipy.integrate import simps
from scipy.io import loadmat
from numpy import trapz
import scipy.fftpack

Using TensorFlow backend.
/home/raquib/anaconda3/envs/tmseeg/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/raquib/anaconda3/envs/tmseeg/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


# Prepare paths of EEG files

In [2]:
paths_clean_hjorth = [
    'data/dataset1/original/sub02/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub02/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub02/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub03/exp03/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub03/exp03/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub03/exp03/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub04/exp02/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub04/exp02/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub04/exp02/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub05/exp02/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub05/exp02/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub05/exp02/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub06/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub06/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub06/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub07/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub07/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub07/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub08/exp03/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub08/exp03/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub08/exp03/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub10/exp02/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub10/exp02/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub10/exp02/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub12/exp03/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub12/exp03/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub12/exp03/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub13/exp02/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub13/exp02/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub13/exp02/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub15/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub15/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub15/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub16/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub16/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub16/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub17/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub17/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub17/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub18/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub18/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub18/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub19/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub19/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub19/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub20/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub20/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub20/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub21/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub21/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub21/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub22/exp01/eeg/SP 110RMT r1/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub22/exp01/eeg/SP 110RMT r2/06-clean-prestimulus-hjorth.mat',
    'data/dataset1/original/sub22/exp01/eeg/SP 110RMT r3/06-clean-prestimulus-hjorth.mat'
]

In [3]:
paths = []
for path in paths_clean_hjorth:
    paths.append({
        'clean_hjorth': path,
        'clean_raw': path[0:-31] + '06-clean-prestimulus.p',
        'hjorth': path[0:-31] + '010-raw-hjorth.mat',
        'raw': path[0:-31] + 'raw.p',
    })
df_paths = pd.DataFrame(paths)

# Basic I/O and conversions.

In [4]:
def read_trials_from_mat(filename):
    x = loadmat(filename)
    mat_trials = x['dat'][0][0][3][0]
    trials = []
    time = np.linspace(-1000, 1000, len(mat_trials[0][0]))
    for mat_trial in mat_trials:
        trials.append(pd.DataFrame({'C3': mat_trial[0], 'C4': mat_trial[1], 'time': time}))
    return trials

def crop_mep_region(mep_frame, crop_start=0.211, crop_end=0.4):
    multiplier = 7499 / 1.4998
    start = int(crop_start * multiplier)
    end = int(crop_end * multiplier)
    mep_cropped = mep_frame.iloc[start:end, :]
    time = mep_frame.iloc[start:end, :]
    return mep_cropped

def get_apb_column_name(mep_frame):
    if 'L APB' in mep_frame:
        return 'L APB'
    else:
        return 'APB'

def open_mep_as_df(path):
    fileMep = open(path, "r+")
    mep_frames = fileMep.read().split('\n\n')
    df_list = []
    for mep_frame in mep_frames:
        df_list.append(pd.read_csv(StringIO(mep_frame), '\t'))
    return df_list

def get_mep_path(eeg_path):
    segments = eeg_path.split('/')
    segments[5] = 'mep'
    segments = segments[:-1]
    mep_path = '/'.join(segments) + '/*.txt'
    mep_path = glob.glob(mep_path)[0]
    return mep_path

def calculate_mep_size(mep_frame, latency, duration):
    mep_cropped = crop_mep_region(mep_frame, latency, latency + duration)
    apb_name = get_apb_column_name(mep_frame)
    max_row = mep_frame.iloc[mep_cropped.idxmax(axis=0)[apb_name]]
    min_row = mep_frame.iloc[mep_cropped.idxmin(axis=0)[apb_name]]
    mep_size = max_row[apb_name] - min_row[apb_name]
    return mep_size, min_row, max_row

def read_latency_duration(eeg_path):
    segments = eeg_path.split('/')
    sub = segments[3]
    exp = segments[4]
    run = segments[6]
    path = 'data/dataset1/original/' + sub + '/' + exp + '/mep/' + run + '/01-ld.csv'
    files = glob.glob(path)
    if len(files) < 1:
        return False
    fileMep = open(path, "r+")
    lines = fileMep.read().split('\n')
    df_ld = []
    for frame_txt in lines:
        if 'Frame' in frame_txt:
            continue
        segments = frame_txt.split('\t')
        if len(segments) >= 4:
            df_ld.append({
                'trial_num': int(segments[0])-1,
                'latency': float(segments[2]),
                'duration': float(segments[3]) - float(segments[2])
            })
    return pd.DataFrame(df_ld)

def calculate_mep_area(mep_frame, latency, duration, plot=False):
    apb_name = get_apb_column_name(mep_frame)
    mep_frame = mep_frame[(mep_frame['s'] >= latency) * (mep_frame['s'] <= latency + duration)]
    amplitude = mep_frame[apb_name]
    amplitude = np.abs(amplitude)
    time_diff = mep_frame['s'].iloc[1] - mep_frame['s'].iloc[0]
    area = trapz(amplitude, dx=time_diff)
    if (plot == True):
        plt.plot(mep_frame['s'], amplitude)
    return area

# Power and phase calculations

In [5]:
def blackman_harris_filter(channel, time, cutoffs, fs, numtaps):
    b = signal.firwin(numtaps, cutoffs, window='blackmanharris', fs=fs)
    filtered = signal.lfilter(b, 1, channel)
    delay = 0.5 * (numtaps - 1) / fs
    df = pd.DataFrame({
        'time': time-delay,
        'channel': filtered
    })
    return df

def butter_bandpass_filter(data, lowcut, highcut, fs, btype, order):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype=btype)
    y = signal.lfilter(b, a, data)
    return y

def get_phase(channel, time, band, fs, filter_type, start_time_ms, stop_time_ms, blackmann_harris_ntaps):
    if filter_type=='butter':
        df_filtered = pd.DataFrame({'channel': butter_bandpass_filter(channel, band[0], band[1], fs, btype='bandpass', order=3), 'time': time})
    else:
        df_filtered = blackman_harris_filter(channel, time, [0.000001, band[0]], fs, numtaps=blackmann_harris_ntaps)
        df_filtered = blackman_harris_filter(df_filtered['channel'].values, df_filtered['time'].values, band[1], fs, numtaps=blackmann_harris_ntaps)    
    
    hilb = signal.hilbert(df_filtered[(df_filtered['time'] > start_time_ms) & (df_filtered['time'] < stop_time_ms)]['channel'])
    phase = np.angle(hilb, deg=True)
    df_phase = pd.DataFrame(phase, columns=['phase'])
    df_phase['time'] = df_filtered[(df_filtered['time'] > start_time_ms) & (df_filtered['time'] < stop_time_ms)]['time'].values
    return df_phase

def get_power(channel, time, fs, crop_start_millis, crop_end_millis, method, filter_type, resample_fs, line_noise_band, blackmann_harris_ntaps):
    # Crop.
    channel = channel[(time > crop_start_millis) & (time < crop_end_millis)].values
    time = time[(time > crop_start_millis) & (time < crop_end_millis)].values
    
    # Resample
    if resample_fs is None:
        resampled = channel
        resampled_time = time
    else:
        ts = time[1] - time[0]
        secs = len(channel) * ts
        secs = secs/1000
        resampled = signal.resample(channel, int(secs*resample_fs))
        resampled_time = np.linspace(time[0], time[-1], len(resampled))
        fs = resample_fs

    # Remove line nosie.
    if filter_type == 'butter':
        resampled = butter_bandpass_filter(resampled, line_noise_band[0], line_noise_band[1], fs, 'bandstop', order=3)
        resampled_time = time
    else:
        df_filtered = blackman_harris_filter(resampled, resampled_time, line_noise_band, fs, numtaps=blackmann_harris_ntaps)
        resampled = df_filtered['channel']
        resampled_time = df_filtered['time']
    
    # PSD.
    if method == 'welch':
        # Welch method
        freq, power = signal.welch(resampled, fs, nperseg=4*fs, nfft=fs)
        df_power = pd.DataFrame({'freq': freq, 'power': power})
    elif method == 'fft':
        # FFT method
        T = 1/fs
        N = fs//2
        yf = scipy.fftpack.fft(resampled, n=N)
        yf = 2 / (N/2) * np.abs(yf[:N//2])
        xf = np.linspace(0, 1/(2*T), N // 2)
        df_power = pd.DataFrame({'freq': xf, 'power': yf})
    elif method == 'pburg':
        # Burgs method
        order = min(len(resampled)-2, int(fs/4))
        p = pburg(resampled, order, sampling=fs, NFFT=fs)
        power = p.psd
        freq = np.linspace(0, fs/2, len(power))
        df_power = pd.DataFrame({'freq': freq, 'power': power})

    df_power = df_power[df_power['freq'] < fs/2]
    df_power['power'] = 10 * np.log10(df_power['power'])
    return df_power

# Diagnose

In [6]:
# start_ms = -150
# end_ms = -1
# fs = 4096
# numtaps = 401
# for filter_type in ['butter', 'blackman']:
#     f, axs = plt.subplots(1,4,figsize=(20,3))
#     channel = channel[(time > start_ms) & (time < end_ms)]
#     time = time[(time > start_ms) & (time < end_ms)]
#     axs[0].plot(time, channel)
#     axs[0].set_title('Waveform')
    
#     df_power = get_power(channel, time, fs, start_ms, end_ms, 'fft', None, None, [48, 52], 801)
#     df_power = df_power[df_power['freq'] < 100]
#     axs[1].plot(df_power['freq'], df_power['power'])
#     axs[1].set_title('Spectrum')
    
#     if filter_type == 'butter':
#         df_filter = pd.DataFrame({'time': time, 'channel': butter_bandpass_filter(channel, 48, 52, fs, 'bandstop', 4)})
#     else:
#         df_filter = blackman_harris_filter(channel, time, [48, 52], fs, numtaps)
#     axs[2].plot(df_filter['time'], df_filter['channel'])
#     axs[2].set_title('Filtered channel')
    
#     df_power = get_power(df_filter['channel'], df_filter['time'], fs, start_ms, end_ms, 'fft', None, None, [48, 52], 801)
#     df_power = df_power[df_power['freq'] < 100]
#     axs[3].plot(df_power['freq'], df_power['power'])
#     axs[3].set_title('Spectrum of filtered')

In [7]:
# order = min(len(df_filter['channel'])-2, int(fs/4))
# p = pburg(resampled, order, sampling=fs, NFFT=fs)
# power = p.psd
# freq = np.linspace(0, fs/2, len(power))
# df_power = pd.DataFrame({'freq': freq, 'power': power})

In [8]:
# ind = 20
# path = df_paths.iloc[ind]['raw']
# trials = pickle.load(open(path, "rb"))
# trial = trials[10]
# time = trial['time']
# channel = trial['C3']

# fs = 4096
# band = [8, 12]
# blackmann_harris_ntaps = 401
# line_noise_band = [48, 52]
# crop_start_millis = -150
# resample_fs = None

# for filter_type in ['butter', 'blackmann']:
#     i = 0
#     f, axs = plt.subplots(1,5,figsize=(16,2))
#     axs[i].plot(time[time < -1], channel[time < -1])
#     i = i + 1
    
#     df_phase = get_phase(channel, time, band=band, fs=fs, filter_type=filter_type, start_time_ms=-750, stop_time_ms=-1, blackmann_harris_ntaps=blackmann_harris_ntaps)
#     axs[i].plot(df_phase['time'], df_phase['phase'])
#     axs[i].set_title("Phase - {}".format(filter_type))
#     i = i + 1
    
#     for method in ['pburg', 'welch', 'fft']:
#         df_power = get_power(channel, time, fs, crop_start_millis=crop_start_millis, crop_end_millis=-1, method=method, filter_type=filter_type, resample_fs=resample_fs, line_noise_band=line_noise_band, blackmann_harris_ntaps=blackmann_harris_ntaps)
#         df_power = df_power[df_power['freq'] < 130]
#         axs[i].plot(df_power['freq'], df_power['power'])
#         axs[i].set_title("Power - {} - {}".format(filter_type, method))
#         i = i + 1
#     plt.tight_layout()
#     plt.show()

# Combine EEG and others

In [9]:
def process_block(path_raw, path_hjorth, path_clean_raw, path_clean_hjorth):
    powers = []
    phases = []
    for artifact_removed, eeg_type, path, fs in zip([False, False, True, True, True, False], ['Raw', 'Hjorth', 'Raw', 'Hjorth', 'Averaged', 'Averaged'], [path_raw, path_hjorth, path_clean_raw, path_clean_hjorth, path_clean_raw, path_raw], [4096, 4096, 2048, 2048, 2048, 4096]):
        # Read EEG file.
        sub = path.split('/')[3]
        exp = path.split('/')[4]
        run = path.split('/')[6]
        if eeg_type == 'Hjorth':
            trials = read_trials_from_mat(path)
        else:
            trials = pickle.load(open(path, "rb"))
            
        # Select channel name.
        if sub in ['sub13', 'sub02']:
            channel = 'C4'
        else:
            channel = 'C3'
            
        # Calculate average of M1 electrodes.
        if eeg_type == 'Averaged':
            new_trials = []
            for trial in trials:
                trial['C3'] = trial[['FC5', 'FC3', 'FC1', 'C5', 'C3', 'C1', 'CP5', 'CP3', 'CP1']].mean(axis=1)
                trial['C4'] = trial[['FC6', 'FC4', 'FC2', 'C6', 'C4', 'C2', 'CP6', 'CP4', 'CP2']].mean(axis=1)
                new_trials.append(trial)
            trials = new_trials
            
        # Read latency duration.
        df_ld = read_latency_duration(path)

        # Calculate powers and phases. And combine the results in a df list.
        for trial_num in df_ld['trial_num'].values:
            if len(trials)-1 < trial_num:
                continue
            trial = trials[trial_num]
            
            # Calculate MEP parameters.
            ld_row = df_ld[df_ld['trial_num'] == trial_num].iloc[0]
            latency = ld_row['latency']
            duration = ld_row['duration']
            mep_frame = open_mep_as_df(get_mep_path(path))[trial_num]
            mep_size, _, _ = calculate_mep_size(mep_frame, latency, duration)
            mep_area = calculate_mep_area(mep_frame, latency, duration)
            
            # PSD and phase calculation.
            for filter_name, filter_code in zip(['Blackmann-Harris', 'Butterworth'], ['butter', 'blackmann-harris']):

                # PSD Calculation.
                for resampled, resample_fs, ntaps_option in zip([True, False], [512, None], [[51, 101], [401, 2001]]):
                    for time, ntaps in zip([-150, -750], [ntaps_option[0], ntaps_option[1]]):
                        for method_name, method_code in zip(['FFT', 'Welch', 'Burg'], ['fft', 'welch', 'pburg']):
                            
                            try:
                                df_power = get_power(trial[channel], trial['time'], fs=fs, crop_start_millis=time, crop_end_millis=-1, method=method_code, filter_type=filter_code, resample_fs=resample_fs, line_noise_band=[48, 52], blackmann_harris_ntaps=ntaps)
                            except:
                                continue
                                
                            for band, fc in zip(['Theta', 'Mu', 'Beta', 'Gamma'], [[3.5, 8], [8, 12], [13, 30], [30, 80]]):
                                powers.append({
                                    'sub': sub,
                                    'exp': exp,
                                    'run': run,
                                    'trial': trial_num,
                                    'ArtifactRemoved': artifact_removed,
                                    'EEG': eeg_type,
                                    'Resampled': resampled,
                                    'Filter': filter_name,
                                    'Time': time,
                                    'Method': method_name,
                                    'Band': band,
                                    'Power': df_power[(df_power['freq'] >= fc[0]) * (df_power['freq'] < fc[1])]['power'].mean(),
                                    'mep_latency': latency,
                                    'mep_duration': duration,
                                    'mep_area': mep_area,
                                    'mep_size': mep_size
                                })
                                
                for band, fc in zip(['Theta', 'Mu', 'Beta', 'Gamma'], [[3.5, 8], [8, 12], [13, 30], [30, 80]]):
                    phase = get_phase(trial[channel], trial['time'], band=fc, fs=fs, filter_type=filter_code, start_time_ms=-750, stop_time_ms=-1, blackmann_harris_ntaps=2001)
                    phase = phase.iloc[-1]['phase'] + 180
                    phases.append({
                        'sub': sub,
                        'exp': exp,
                        'run': run,
                        'trial': trial_num,
                        'ArtifactRemoved': artifact_removed,
                        'EEG': eeg_type,
                        'Filter': filter_name,
                        'Band': band,
                        'Phase': phase,
                        'mep_latency': latency,
                        'mep_duration': duration,
                        'mep_area': mep_area,
                    })
    return (powers, phases)

In [10]:
# Run analysis
paths = [list(a) for a in zip(df_paths['raw'].values, df_paths['hjorth'].values, df_paths['clean_raw'].values, df_paths['clean_hjorth'].values)]
num_cores = multiprocessing.cpu_count() - 2
results = Parallel(n_jobs=num_cores)(delayed(process_block)(path_raw, path_hjorth, path_clean_raw, path_clean_hjorth) for path_raw, path_hjorth, path_clean_raw, path_clean_hjorth in tqdm(paths))

# Populate absolute trial numbers and save to csv

In [11]:
df_powers = []
df_phases = []
for powers, phases in tqdm(results):
    for power in powers:
        df_powers.append(power)
    for phase in phases:
        df_phases.append(phase)

df_powers = pd.DataFrame(df_powers)
df_phases = pd.DataFrame(df_phases)

In [12]:
def_values_power = {
    'ArtifactRemoved': False, 
    'EEG': 'Raw', 
    'Resampled': False, 
    'Filter': 'Blackmann-Harris', 
    'Time': -750, 
    'Method': 'Welch'
}
def_values_phase = {
    'ArtifactRemoved': False, 
    'EEG': 'Raw',
    'Filter': 'Butterworth', 
}

In [22]:
df_powers_new = []
keys = list(def_values_power.keys())
for sub in tqdm(df_powers['sub'].unique()):
    df_sub = df_powers[df_powers['sub'] == sub]
    for val1 in df_powers[keys[0]].unique():
        for val2 in df_powers[keys[1]].unique():
            for val3 in df_powers[keys[2]].unique():
                for val4 in df_powers[keys[3]].unique():
                    for val5 in df_powers[keys[4]].unique():
                        for val6 in df_powers[keys[5]].unique():
                            for val7 in df_powers['Band'].unique():
                                df_powers2 = df_sub[df_sub['Band'] == val7]
                                df_powers2 = df_powers2[df_powers2[keys[5]] == val6]
                                df_powers2 = df_powers2[df_powers2[keys[4]] == val5]
                                df_powers2 = df_powers2[df_powers2[keys[3]] == val4]
                                df_powers2 = df_powers2[df_powers2[keys[2]] == val3]
                                df_powers2 = df_powers2[df_powers2[keys[1]] == val2]
                                df_powers2 = df_powers2[df_powers2[keys[0]] == val1]
                                i = 1
                                for idx, row in df_powers2.iterrows():
                                    row['trial_abs'] = i
                                    df_powers_new.append(row)
                                    i = i + 1
df_powers_new = pd.DataFrame(df_powers_new)

In [23]:
df_phases_new = []
keys = list(def_values_phase.keys())
for sub in tqdm(df_phases['sub'].unique()):
    df_sub = df_phases[df_phases['sub'] == sub]
    for val1 in df_phases[keys[0]].unique():
        for val2 in df_phases[keys[1]].unique():
            for val3 in df_phases[keys[2]].unique():
                for val4 in df_phases['Band'].unique():
                    df_phases2 = df_sub[df_sub[keys[0]] == val1]
                    df_phases2 = df_phases2[df_phases2[keys[1]] == val2]
                    df_phases2 = df_phases2[df_phases2[keys[2]] == val3]
                    df_phases2 = df_phases2[df_phases2['Band'] == val4]
                    i = 1
                    for idx, row in df_phases2.iterrows():
                        row['trial_abs'] = i
                        df_phases_new.append(row)
                        i = i + 1
df_phases_new = pd.DataFrame(df_phases_new)

In [24]:
df_powers_new.to_csv('164-d1-powers.csv')
df_phases_new.to_csv('164-d1-phases.csv')

# Test

In [16]:
# path_clean_raw = df_paths['raw'].values[53]

# powers = []
# phases = []
# for artifact_removed, eeg_type, path, fs in zip([False], ['Averaged'], [path_clean_raw], [4096]):
#     # Read EEG file.
#     sub = path.split('/')[3]
#     exp = path.split('/')[4]
#     run = path.split('/')[6]
#     if eeg_type == 'Hjorth':
#         trials = read_trials_from_mat(path)
#     else:
#         trials = pickle.load(open(path, "rb"))

#     # Select channel name.
#     if sub in ['sub13', 'sub02']:
#         channel = 'C4'
#     else:
#         channel = 'C3'

#     # Calculate average of M1 electrodes.
#     if eeg_type == 'Averaged':
#         new_trials = []
#         for trial in trials:
#             trial['C3'] = trial[['FC5', 'FC3', 'FC1', 'C5', 'C3', 'C1', 'CP5', 'CP3', 'CP1']].mean(axis=1)
#             trial['C4'] = trial[['FC6', 'FC4', 'FC2', 'C6', 'C4', 'C2', 'CP6', 'CP4', 'CP2']].mean(axis=1)
#             new_trials.append(trial)
#         trials = new_trials

#     # Read latency duration.
#     df_ld = read_latency_duration(path)

#     # Calculate powers and phases. And combine the results in a df list.
#     for trial_num in tqdm(df_ld['trial_num'].values):
# #         if len(trials)-1 < trial_num:
# #             continue
#         trial = trials[trial_num]

#         # Calculate MEP parameters.
#         ld_row = df_ld[df_ld['trial_num'] == trial_num].iloc[0]
#         latency = ld_row['latency']
#         duration = ld_row['duration']
#         mep_frame = open_mep_as_df(get_mep_path(path))[trial_num]
#         mep_size, _, _ = calculate_mep_size(mep_frame, latency, duration)
#         mep_area = calculate_mep_area(mep_frame, latency, duration)

#         # PSD and phase calculation.
#         for filter_name, filter_code in zip(['Butterworth'], ['butter']):

#             # PSD Calculation.
#             for resampled, resample_fs, ntaps_option in zip([False], [None], [[401, 2001]]):
#                 for time, ntaps in zip([-750], [ntaps_option[1]]):
#                     for method_name, method_code in zip(['FFT', 'Welch', 'Burg'], ['fft', 'welch', 'pburg']):

# #                         try:
#                         df_power = get_power(trial[channel], trial['time'], fs=fs, crop_start_millis=time, crop_end_millis=-1, method=method_code, filter_type=filter_code, resample_fs=resample_fs, line_noise_band=[48, 52], blackmann_harris_ntaps=ntaps)
# #                         except:
# #                             continue

#                         for band, fc in zip(['Theta', 'Mu', 'Beta', 'Gamma'], [[3.5, 8], [8, 12], [13, 30], [30, 80]]):
#                             powers.append({
#                                 'sub': sub,
#                                 'exp': exp,
#                                 'run': run,
#                                 'trial': trial_num,
#                                 'ArtifactRemoved': artifact_removed,
#                                 'EEG': eeg_type,
#                                 'Resampled': resampled,
#                                 'Filter': filter_name,
#                                 'Time': time,
#                                 'Method': method_name,
#                                 'Band': band,
#                                 'Power': df_power[(df_power['freq'] >= fc[0]) * (df_power['freq'] < fc[1])]['power'].mean(),
#                                 'mep_latency': latency,
#                                 'mep_duration': duration,
#                                 'mep_area': mep_area
#                             })

#             for band, fc in zip(['Theta', 'Mu', 'Beta', 'Gamma'], [[3.5, 8], [8, 12], [13, 30], [30, 80]]):
#                 phase = get_phase(trial[channel], trial['time'], band=fc, fs=fs, filter_type=filter_code, start_time_ms=-750, stop_time_ms=-1, blackmann_harris_ntaps=2001)
#                 phase = phase.iloc[-1]['phase'] + 180
#                 phases.append({
#                     'sub': sub,
#                     'exp': exp,
#                     'run': run,
#                     'trial': trial_num,
#                     'ArtifactRemoved': artifact_removed,
#                     'EEG': eeg_type,
#                     'Filter': filter_name,
#                     'Band': band,
#                     'Phase': phase,
#                     'mep_latency': latency,
#                     'mep_duration': duration,
#                     'mep_area': mep_area,
#                 })

In [17]:
# df_powers = pd.DataFrame(powers)
# df_powers.head()

In [18]:
# df_paths['clean_raw'].values[53]